#### Imports

In [3]:
import tensorflow as tf                  
import tensorflow.nn as nn               
from tensorflow import keras             
import tensorflow.keras.layers as layers 
import numpy as np                       
from sklearn.model_selection import train_test_split   
from tensorflow.keras.metrics import categorical_accuracy
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, Dense, Dropout, ELU, Flatten, MaxPool2D
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.initializers import lecun_uniform
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt

#### Preprocessing

In [4]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")


In [5]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))


Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [6]:
# rearange input dimensions
# model expects (batch, seq, features) but currently we have (batch, features, seq)

X_train_valid = np.transpose(X_train_valid, (0, 2, 1))
X_test = np.transpose(X_test, (0, 2, 1))

print("training data:", X_train_valid.shape)
print("testing data:", X_test.shape)

training data: (2115, 1000, 22)
testing data: (443, 1000, 22)


In [7]:
# re-assign labels to be between 0 and 3
y_train_valid[y_train_valid==769] = 0  # cue onset left
y_train_valid[y_train_valid==770] = 1  # cue onset right
y_train_valid[y_train_valid==771] = 2  # cue onset foot
y_train_valid[y_train_valid==772] = 3  # cue onset tongue

y_test[y_test==769] = 0  # cue onset left
y_test[y_test==770] = 1  # cue onset right
y_test[y_test==771] = 2  # cue onset foot
y_test[y_test==772] = 3  # cue onset tongue

In [8]:
# split the training and validation
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.2, random_state=12345)

In [9]:
# add extra dimension for grayscale images
X_train_grayscale = np.expand_dims(X_train, axis=-1)
X_valid_grayscale = np.expand_dims(X_valid, axis=-1)
X_test_grayscale = np.expand_dims(X_test, axis=-1)

print("training images:", X_train_grayscale.shape)
print("training labels:", y_train.shape)
print("validation images:", X_valid_grayscale.shape)
print("validation labels:", y_valid.shape)
print("test images:", X_test_grayscale.shape)
print("test labels:", y_test.shape)

training images: (1692, 1000, 22, 1)
training labels: (1692,)
validation images: (423, 1000, 22, 1)
validation labels: (423,)
test images: (443, 1000, 22, 1)
test labels: (443,)


In [10]:
# construct the dataset from the NumPy arrays
X_train_grayscale = tf.data.Dataset.from_tensor_slices((X_train_grayscale, y_train))
X_valid_grayscale = tf.data.Dataset.from_tensor_slices((X_valid_grayscale, y_valid))
X_test_grayscale = tf.data.Dataset.from_tensor_slices((X_test_grayscale, y_test))

# batch and shuffle
X_train_grayscale = X_train_grayscale.shuffle(1000).batch(32)
X_valid_grayscale = X_valid_grayscale.shuffle(300).batch(32)
X_test_grayscale = X_test_grayscale.shuffle(300).batch(32)


# construct the dataset from the NumPy arrays
X_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
X_valid = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))
X_test = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# batch and shuffle
X_train = X_train.shuffle(1024).batch(32)
X_valid = X_valid.shuffle(1024).batch(32)
X_test = X_test.shuffle(1024).batch(32)

2023-03-09 22:42:13.374405: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Default Project

#### 1. Optimize the classification accuracy for subject 1. Does it help to train across all subjects?

#### 2. Optimize the classification accuracy across all subjects. How does the classifier do? Do you notice any interesting trends?

#### 3. Evaluate the classification accuracy as a function of time (e.g., does it increase as you have data over longer periods of time? how much time is required to get a reasonable classification accuracy?)

## Deeper Exploration and Analysis into other architectures